In [ ]:
import nibabel as nib
from skimage.measure import marching_cubes
import numpy as np
import open3d as o3d
import pyvista
import matplotlib.pyplot as plt
import vtkmodules
import time

In [ ]:
def pad_edge_list(edges):
    padding = np.ones(edges.shape[0], int)*3
    edges_w_padding = np.vstack((padding, edges.T)).T
    return edges_w_padding

In [ ]:
path = "C:/Users/aorhu/Masaüstü/body_mask.nii.gz"

In [ ]:
body_segment = nib.load(path)
body_segment_data = body_segment.get_fdata()

In [ ]:
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1)

In [ ]:
verts

In [ ]:
len(verts)

In [ ]:
len(faces)

In [ ]:
faces

In [ ]:
verts = verts/np.array(body_segment_data.shape) 

### open3d quadric_decimation

In [ ]:
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))


start = time.time()
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, 25000)
end = time.time()

print("The time of execution of above program is :",(end-start) , "s")
print("The time of execution of above program is :",(end-start) * 10**3, "ms")

In [ ]:
#### to save the object
o3d.io.write_triangle_mesh("test.off", decimated_mesh, print_progress=True )

In [ ]:
decimated_mesh = o3d.io.read_triangle_mesh('test.off')

In [ ]:
decimated_mesh

In [ ]:
mesh.triangles

In [ ]:
decimated_mesh.triangles

In [ ]:
np.asarray(decimated_mesh.vertices)[:,1]= np.asarray(decimated_mesh.vertices)[:,1]+1

In [ ]:
original_mesh_paint = np.asarray([0,200,220])/255.0
decimated_mesh_paint = np.asarray([230,200,110])/255.0
mesh.paint_uniform_color(original_mesh_paint)
decimated_mesh.paint_uniform_color(decimated_mesh_paint)

In [ ]:
mesh.compute_vertex_normals()
decimated_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([decimated_mesh],mesh_show_back_face=True,mesh_show_wireframe=True)

#### clustering

In [ ]:
voxel_size = max(mesh.get_max_bound() - mesh.get_min_bound()) / 64
print(f'voxel_size = {voxel_size:e}')
mesh_smp = mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)

In [ ]:
mesh_smp.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_smp],mesh_show_back_face=True,mesh_show_wireframe=True)

#### surface area and volume comparison

In [ ]:
mesh.get_surface_area()

In [ ]:
decimated_mesh.get_surface_area()

### trimesh repair

In [ ]:
import trimesh

In [ ]:
tr_mesh= trimesh.Trimesh(vertices=verts, faces=faces)

In [ ]:
tr_mesh

In [ ]:
len(trimesh.repair.broken_faces(tr_mesh))

In [ ]:
trimesh.repair.fill_holes(tr_mesh)

In [ ]:
tr_mesh

### pyvista decimation

In [ ]:
edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)
lines = np.concatenate((np.int32(2*np.ones((edges.shape[0],1))), edges), 1)
mesh = pyvista.PolyData(verts, pad_edge_list(faces))

In [ ]:
#decimated_mesh = mesh.decimate(0.5)
pro_decimated = mesh.decimate_pro(0.2, preserve_topology=False)

In [ ]:
pyvista.set_plot_theme("document")
#plotter = pyvista.Plotter(shape=(1, 1), window_size=[1000, 500], border=False)
plotter = pyvista.Plotter(notebook=False)
plotter.add_mesh(pro_decimated, render_points_as_spheres=False, color="lightcoral", show_edges=True, line_width=0.3, edge_color='black', point_size=1)
plotter.camera.zoom(3)
plotter.set_position((0,-7,0))

In [ ]:
plotter.show(auto_close=False)

In [ ]:
len(mesh.faces)

In [ ]:
len(pro_decimated.faces)

### VTK decimation (not working)

In [ ]:
import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkCommonDataModel import vtkPolyData
from vtkmodules.vtkFiltersCore import (
    vtkDecimatePro,
    vtkTriangleFilter
)
import vtk

In [ ]:
mesh = vtkPolyData()

In [ ]:
mesh.SetPoints(vtk.vtkPoints(verts))

In [ ]:
verts.shape[0]

In [ ]:
vpoints = vtk.vtkPoints()
vpoints.SetNumberOfPoints(verts.shape[0])
for i in range(verts.shape[0]):
    vpoints.SetPoint(i, verts[i])
mesh = vtk.vtkPolyData()
mesh.SetPoints(vpoints)